# Controlled Bouncing Ball

Example taken from: ProbReach paper

```
mass R = ?, C = ?, H = ?

repeat {
    if H<= 0 {
        mode = 1
    }
    else {
        mode = -1
    }
}
```

Height and velocity evolve according to:
$$ \frac{d \textit{H}}{dt} = \begin{cases} v & \text{if mode = -1} \\
                                             v & \text{if mode = 1}  \end{cases} $$

$$ \frac{d \textit{v}}{dt} = \begin{cases} -g & \text{if mode = -1} \\
                                              - g - \frac{Rv + \frac{H}{C}}{m} & \text{if mode = 1}  \end{cases} $$


We try to synthesize R and H/C in order to maximize the probability that the ball reaches H >=7 in mode 1 after making one bounce

## SOGA

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
print(sys.path)

from sogaPreprocessor import *
from producecfg import *
from smoothcfg import *
from libSOGA import *
from time import time
import matplotlib.pyplot as plt
import numpy as np
import torch

['/home/romina/PhD/DEGAS/DeGAS/src/REACHABILITY', '/home/romina/anaconda3/lib/python39.zip', '/home/romina/anaconda3/lib/python3.9', '/home/romina/anaconda3/lib/python3.9/lib-dynload', '', '/home/romina/anaconda3/lib/python3.9/site-packages', '/home/romina/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/romina/.ipython', '/home/romina/PhD/DEGAS/DeGAS/src']


In [ ]:
from optimization import *
torch.set_default_dtype(torch.float64)

: 

In [ ]:
soga_code = """
array[36] H;
array[36] V;
array[36] M; 

/* Initial state */
currH = gauss(9., 1.);   
mode = -1.;
currV = 0.;                 

dt = 0.03;            

for i in range(35) {

    /* Save current state */
    H[i] = currH;
    M[i] = mode;
    V[i] = currV;

    /* Continuous dynamics */

    temp = currV * dt;
    newH = currH + temp + gauss(0., 0.1);

    if mode < 0 {
        /* falling down */
        temp = -9.8 * dt;
        newV = currV + temp + gauss(0., 0.1);
    } else {
        /* going up */
        temp = -9.8 * dt;

        spring = _R*currV;
        temp2 = _C*currH;
        spring = spring + temp2;
        spring = spring * dt; 
        spring = 0.14 * spring; 
        newV = currV + temp - spring + gauss(0., 0.1);

    } end if;

    currH = newH;
    currV = newV;

    /* Mode switching */
    if mode < 0 {
        if currH <= 0. {
            mode = 1;
        } else {
            skip;
        } end if;
    } else {
        if currT > 0. {
            mode = -1;
        } else {
            skip;
        } end if;
    } end if;

} end for;

/* Save final state */
H[35] = currH;
M[35] = mode;
V[35] = currV;

"""

compiledFile = compile2SOGA_text(soga_code)
cfg = produce_cfg_text(compiledFile)
smooth_cfg(cfg)

# initialize parameters
params = {'R':  5., 'C':400.}
params_dict = initialize_params(params)  

# computes SOGA output
output_dist = start_SOGA(cfg, params_dict)

#plot
y_init = output_dist.gm.mean()[:35].detach()
plt.plot(range(35), y_init, lw=3, label='SOGA w true params')

v_init = output_dist.gm.mean()[36:70].detach()
plt.plot(range(36, 70, 1), v_init, lw=3, label='SOGA w true params')
# Points to which the lines should be drawn
#points = [(6, 20), (18, 20), (24, 20)]

# Plot the points on the graph
#for point in points:
    #plt.plot(point[0], point[1], 'ro')  # Red dot at each point
#plot_traj_set(orig_traj, single_traj=10, color='red', label='orig')